In [1]:
from __future__ import print_function 
import os, sys, h5py
import numpy as np
from six.moves import cPickle
from collections import OrderedDict

import tensorflow as tf
sys.path.append('..')
from tfomics import layers, utils, init, learn, explore
from tfomics import neuralnetwork as nn
from tfomics.build_network import *

# import models
from model_zoo import simple_connectomics_model_explore

In [2]:
filename = 'processed_dataset.hdf5'
data_path = '/home/peter/Code/tensorflow/data'
filepath = os.path.join(data_path,filename)

group_name = ['processed_data']
dataset = h5py.File(filepath,'r')
%time dtf = np.array(dataset['/'+group_name[0]+'/dtf'])
ltf = np.array(dataset['/'+group_name[0]+'/ltf'])
dtf_crossval = np.array(dataset['/'+group_name[0]+'/dtf_crossval'])
ltf_crossval = np.array(dataset['/'+group_name[0]+'/ltf_crossval'])

X_train = dtf.transpose([0,2,3,1])
y_train = ltf
X_valid = dtf_crossval.transpose([0,2,3,1])
y_valid = ltf_crossval


CPU times: user 448 ms, sys: 1.01 s, total: 1.46 s
Wall time: 1.46 s


In [3]:
# get shapes
num_data, seq_length, _, dim = X_train.shape
input_shape=[None, seq_length, 1, dim]
num_labels = y_train.shape[1]   # number of labels (output units)

# load flexible models
model_layers, placeholders, optimization = simple_connectomics_model_explore.model(input_shape, num_labels)

# build neural optimizer 
save_name = 'test'
save_path = utils.make_directory('../results', 'explore_simple_connectomics')
filepath = os.path.join(save_path, save_name)
optimizer = explore.NeuralOptimizer(model_layers, placeholders, optimization, filepath=filepath)


making directory: ../results/explore_simple_connectomics


In [4]:

# optimize model parameters
train = {'inputs': X_train, 'targets': y_train, 'keep_prob': 0.8, 'is_training': True}
valid = {'inputs': X_valid, 'targets': y_valid, 'keep_prob': 1, 'is_training': False}        
optimizer.optimize(train, valid, num_trials=20, num_epochs=5, batch_size=128, verbose=0)


---------------------------------------------------------
Running baseline model

Model layers:
name: input
name: conv1
pool_size: 4
filter_size: 5
num_filters: 20
name: conv2
pool_size: 4
filter_size: 5
num_filters: 20
name: dense1
num_units: 120
name: dense2
num_units: 2

Optimization:
learning_rate: 0.001
l2: 1e-06

    loss = 0.356943200971
    took 191.018948793 seconds

---------------------------------------------------------
trial 1 out of 20
---------------------------------------------------------

Model layers:
name: input
name: conv1
pool_size: 4
filter_size: 10
num_filters: 10
name: conv2
pool_size: 8
filter_size: 16
num_filters: 45
name: dense1
num_units: 230
name: dense2
num_units: 2

Optimization:
learning_rate: 0.00328777179477
l2: 1.03354070345e-08

Results:
loss = 0.359514427188
took 193.184247971 seconds

---------------------------------------------------------
trial 2 out of 20
---------------------------------------------------------

Model layers:
name: input
na

In [ ]:
# save optimal model
optimizer.print_optimal_model()